In [12]:
# -*- coding: utf-8 -*- 
import os
import numpy as np
import pandas as pd

from pathlib import Path

from hbayesdm.models import dd_hyperbolic

import nibabel as nib
# import nipype as nip
# import nilearn as nil

from bids import BIDSLayout, BIDSValidator
from tqdm import tqdm

In [13]:
data_root = Path('piva_dd')
fmriprep_root = Path('output/fmriprep')

In [14]:
layout = BIDSLayout(data_root, derivatives=True)
#layout.add_derivatives(fmriprep_root)

/home/cheoljun/anaconda3/lib/python3.7/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)
/home/cheoljun/anaconda3/lib/python3.7/site-packages/bids/layout/layout.py:591: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations (['piva_dd/derivatives']). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "PipelineDescription": {"Name": "Example pipeline"}}
  json.dumps(EXAMPLE_DERIVATIVES_DESCRIPTION))


In [15]:
subjN = len(layout.get(target='subject', return_type='id'))

In [16]:
frmi_subject_run = {}
# simple structure for fmri_subject_run
# fmri_subject_run: {
#     subj: {
#         niis: [run1, run2, run3],
#         events: [run1, run2, run3]
#     }
# }

for subject_id in tqdm(layout.get(target='subject', return_type='id')):
    if subject_id not in frmi_subject_run.keys():
        frmi_subject_run[subject_id] = {
            #'niis': layout.derivatives['fMRIPrep'].get(subject=subject_id, return_type='file', suffix='bold', extension='nii.gz'),
            'events': layout.get(subject=subject_id, return_type='file', suffix='events', extension='tsv')
        }
    else: # subjID is already exists..
        print(f'error! {subject_id}')
        break

100%|██████████| 20/20 [00:00<00:00, 159.65it/s]


In [17]:
    
def function_0(df):    
    
    new_df = {}
    new_df['onset'] = []
    new_df['duration'] = []
    new_df['delay_later'] = []
    new_df['delay_sooner'] = []
    new_df['amount_later'] = [] 
    new_df['amount_sooner'] = []
    new_df['choice'] = []
    
    for _,row in df[df['agent']==0].iterrows():
        new_df['onset'].append(row['onset'])
        new_df['duration'].append(row['duration'])
        if row['delay_left'] >= [row['delay_right']]:
            new_df['delay_later'].append(row['delay_left'])
            new_df['delay_sooner'].append(row['delay_right'])
            new_df['amount_later'].append(row['money_left'])
            new_df['amount_sooner'].append(row['money_right'])
            new_df['choice'].append(1 if row['choice'] == 1 else 0)
        else:
            new_df['delay_later'].append(row['delay_right'])
            new_df['delay_sooner'].append(row['delay_left'])
            new_df['amount_later'].append(row['money_right'])
            new_df['amount_sooner'].append(row['money_left'])
            new_df['choice'].append(1 if row['choice'] == 2 else 0)
    
    return pd.DataFrame(new_df)
        
    

In [18]:
df_all = []

for subject_id in layout.get(target='subject', return_type='id'):
    for i, df_path in enumerate(frmi_subject_run[subject_id]['events']):
        df = pd.read_table(df_path)
        df = function_0(df)
        df['subjID'] = int(subject_id)
        df['run'] = i + 1
        df_all.append(df)
    
df_all = pd.concat(df_all)

In [38]:
'''
model = dd_hyperbolic(data=df_all, ncore=4)
print(model.all_ind_pars)
params = pd.DataFrame({'k_mu': [model.fit[f'k[{i}]'].mean() for i in range(1,subjN+1)],
        'k_sigma' : [model.fit[f'k[{i}]'].std() for i in range(1,subjN+1)],
        'beta_mu' : [model.fit[f'beta[{i}]'].mean() for i in range(1,subjN+1)],
        'beta_sigma' : [model.fit[f'beta[{i}]'].mean() for i in range(1,subjN+1)]
            })
                     
params.to_csv('dd_hyperbolic_params.tsv', sep='\t', index=False)
'''

In [39]:
params = pd.read_csv('dd_hyperbolic_params.tsv', sep='\t')
params['subjID'] = params.index + 1

In [40]:
def get_utility(k_mu,k_sigma, beta_mu, beta_sigma,
                delay_later,delay_sooner,amount_later,amount_sooner):
    k = k_mu 
    ev_later   = amount_later / (1 + k * delay_later)
    ev_sooner  = amount_sooner/ (1 + k * delay_sooner)
    utility = ev_later - ev_sooner
    
    return utility

In [41]:
save_root = Path('output/latent')


In [53]:
utilities = [get_utility(float(params[params['subjID'] == row['subjID']]['k_mu']),
                         float(params[params['subjID'] == row['subjID']]['k_sigma']),
                         float(params[params['subjID'] == row['subjID']]['beta_mu']),
                         float(params[params['subjID'] == row['subjID']]['beta_sigma']),
                         float(row['delay_later']),
                         float(row['delay_sooner']),
                         float(row['amount_later']),
                         float(row['amount_sooner']))
             for _, row in df_all.iterrows()]


In [56]:
df_all['utilities'] = utilities
params.to_csv(save_root / 'dd_hyperbolic_latent.tsv', sep='\t', index=False)

In [58]:
df_all.head()

,onset,duration,delay_later,delay_sooner,amount_later,amount_sooner,choice,subjID,run,utilities
0,218.071854,5.0,132.0,21.0,16.31,12.65,0,1,1,-2.600491
1,231.077068,5.0,154.0,16.0,27.53,21.56,0,1,1,-6.230580
2,242.080367,5.0,108.0,96.0,15.77,5.87,1,1,1,5.424555
3,255.083959,5.0,90.0,84.0,12.06,9.17,1,1,1,1.610895
4,270.088602,5.0,115.0,31.0,28.26,4.01,1,1,1,12.243544
